In [27]:
from datetime import datetime, timedelta
import time
import pandas as pd

# Data cleaning

Functie om de data op te kuisen

In [28]:
def clean_data(df):
    # Verwijder de kolom 'ranking', deze is nutteloos
    df.drop('ranking', axis=1, inplace=True)

    # Verwijder rijen met null waarden, deze zijn fout geregistreerd in de DB
    df.dropna(inplace=True)
        
    # Verwijder rijen waarvan tijdformaat nie overeenkomt met xx:xx:xx
    time_pattern = r'^\d{2}:\d{2}:\d{2}$'

    df = df[
        df["startTime"].str.match(time_pattern, na=False) & 
        df["rLength"].str.match(time_pattern, na=False)
    ].copy()

    # rLength omzetten naar seconden -> duration_sec
    # dateDiff omzetten naar datetime
    df['duration_sec'] = pd.to_timedelta(df['rLength']).dt.total_seconds()
    df['date'] = pd.to_datetime(df["dateDiff"]).dt.date

    # rateInK omzetten naar int -> viewers
    df['viewers'] = df['rateInK'].apply(lambda x: int(''.join(str(x).split('.'))))

    #startTime: 24:30:00 omzetten naar 00:30:00 en dateDiff 1 dag verhogen
    def fix_next_day(rij):
        time_parts = rij['startTime'].split(':')
        if int(time_parts[0]) >= 24:
            time_parts[0] = str(int(time_parts[0]) - 24).zfill(2)
            rij['date'] += timedelta(days=1)
        rij['startTime'] = ':'.join(time_parts)
        return rij

    df = df.apply(fix_next_day, axis=1)

    # timestamp kolom toevoegen op basis van dateDiff en startTime
    df['timestamp'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['startTime'].astype(str))

    # Live: boolean maken, of getal houden?? 28 > 7 > 1 > 0 heeft groter getal ook correlatie met kijkcijfer???
    # Beslissing: Live kolom laten als int

    # Hour year month day toevoegen
    df['hour'] = pd.to_datetime(df['startTime']).dt.hour
    df['year'] = pd.to_datetime(df['date']).dt.year
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['day'] = pd.to_datetime(df['date']).dt.day

    # Onnodige kolommen verwijderen
    df.drop(['startTime', 'rLength', 'rateInK'], axis=1, inplace=True)

    df = df[['timestamp', 'date', 'year', 'month', 'day', 'hour', 'channel', 'description', 'duration_sec', 'live', 'viewers']]

    df.rename(columns={'description': 'program'}, inplace=True)

    return df

Data opkuisen

In [29]:
kijkcijfers = pd.read_csv('./data/raw/tv_kijkcijfers_raw.csv')
kijkcijfers = clean_data(kijkcijfers)
kijkcijfers.sample(5)

C:\Users\dylan\AppData\Local\Temp\ipykernel_17732\3836137169.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hour'] = pd.to_datetime(df['startTime']).dt.hour


,timestamp,date,year,month,day,hour,channel,program,duration_sec,live,viewers
5051,2017-06-12 19:00:05,2017-06-12,2017,6,12,19,EEN,HET 7 UUR-JOURNAAL,2511.0,0,834071
60547,2024-12-29 21:10:24,2024-12-29,2024,12,29,21,VTM,HET CONCLAAF VAN 2024,3239.0,28,338017
11008,2018-04-05 20:41:14,2018-04-05,2018,4,5,20,CANVAS,VB. C3 UEFA 1/4F,1032.0,0,196268
27832,2020-07-26 13:00:02,2020-07-26,2020,7,26,13,EEN,HET 1 UUR-JOURNAAL,1924.0,0,546728
59546,2024-11-09 19:56:24,2024-11-09,2024,11,9,19,VTM,EXPEDITIE PAIRI DAIZA,1698.0,28,319191


# Weerdata samenvoegen met kijkcijferdata

In [30]:
weer_data = pd.read_csv('./data/raw/weerdata.csv')
weer_data['datetime'] = pd.to_datetime(weer_data['time'])
weer_data['date'] = weer_data['datetime'].dt.date
weer_data['hour'] = weer_data['datetime'].dt.hour
weer_data['year'] = weer_data['datetime'].dt.year
weer_data['month'] = weer_data['datetime'].dt.month
weer_data['day'] = weer_data['datetime'].dt.day

weer_data = weer_data.drop(columns=['time', 'day_of_week'])

weer_data.sample(5)

,temperature,weather_code,precipitation,rain,snowfall,cloudcover,windspeed,hour,month,year,datetime,date,day
77983,10.2,53.0,0.9,0.9,0.0,100.0,42.1,7,1,2024,2024-01-22 07:00:00,2024-01-22,22
14451,5.0,0.0,0.0,0.0,0.0,19.0,7.7,3,10,2016,2016-10-23 03:00:00,2016-10-23,23
25393,4.5,51.0,0.2,0.2,0.0,100.0,20.3,1,1,2018,2018-01-22 01:00:00,2018-01-22,22
78167,8.1,3.0,0.0,0.0,0.0,92.0,17.6,23,1,2024,2024-01-29 23:00:00,2024-01-29,29
38072,14.6,0.0,0.0,0.0,0.0,0.0,10.7,8,7,2019,2019-07-04 08:00:00,2019-07-04,4


Weerdata mergen op datum en uur

In [31]:
kijkcijfers_weer = pd.merge(kijkcijfers, weer_data, on=['date', 'hour'], how='left')

kijkcijfers_weer = kijkcijfers_weer[['timestamp', 'channel', 'program', 'duration_sec', 'live', 'viewers', 'weather_code', 'temperature', 'rain', 'windspeed', 'snowfall', 'precipitation']]

kijkcijfers_weer.sample(5)

,timestamp,channel,program,duration_sec,live,viewers,weather_code,temperature,rain,windspeed,snowfall,precipitation
491,2016-10-25 13:00:05,EEN,HET 1 UUR-JOURNAAL,1818.0,0,387803,3.0,12.2,0.0,1.8,0.0,0.0
9737,2018-02-04 10:45:14,CANVAS,SPORZA,2816.0,0,255358,2.0,-0.2,0.0,11.1,0.0,0.0
4401,2017-05-13 20:13:28,EEN,"BLANCHE, EEN PORTRET",1095.0,0,518488,1.0,14.2,0.0,8.4,0.0,0.0
30577,2020-12-30 22:40:13,VTM,"HELDEN VAN HIER, DOOR HET VUUR",3518.0,0,232800,3.0,3.3,0.0,13.9,0.0,0.0
21819,2019-10-04 19:00:06,EEN,HET 7 UUR-JOURNAAL,2736.0,0,858362,51.0,12.9,0.1,16.3,0.0,0.1


Kijk of er nu nullrows zijn

In [32]:
print(f"Aantal null-rows: \n{kijkcijfers_weer.isnull().sum()}")

Aantal null-rows: 
timestamp         0
channel           0
program           0
duration_sec      0
live              0
viewers           0
weather_code     20
temperature      20
rain             20
windspeed        20
snowfall         20
precipitation    20
dtype: int64


De weerdata heeft 20 missende rijen data => verwijder deze

In [33]:
kijkcijfers_weer.dropna(inplace=True)

Gecombineerde data wegschrijven

In [34]:
kijkcijfers_weer.to_csv('./data/processed/tv_kijkcijfers_weer.csv', index=False)

In [44]:
kijkcijfers_weer['viewers'].describe().apply(lambda x: format(x, ',.2f'))

count       60,508.00
mean       446,055.26
std        278,183.65
min         15,887.00
25%        229,910.00
50%        360,262.00
75%        606,146.00
max      2,494,114.00
Name: viewers, dtype: object